<a href="https://colab.research.google.com/github/Matheus5S/Python/blob/main/An%C3%A1lise_2_Trade_Quantitativo_Escolhendo_a_Estrat%C3%A9gia_%C3%93tima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#@title Rodar no colab para instalar pacotes necessários {display-mode: "form"}

# This code will be hidden when the notebook is loaded.
# Run This Cell for Colab
# Just instaling the required packages

!pip install yfinance
!pip install vectorbt
!pip install quantstats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 430 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 526 kB 5.1 MB/s 
     |████████████████████████████████| 292 kB 14.4 MB/s 
     |████████████████████████████████| 1.6 MB 22.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 251 kB/s 


In [ ]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Definindo parâmetros

In [ ]:
# Análise de um Ativo Americano
codigo = 'AAPL'
ativos_comparacao = [codigo, '^GSPC', 'AMZN', 'NFLX']

data_inicial = '2017-01-01'  
data_final = '2018-01-01'

# Os preços de fechamentos dos ativos
vbt.YFData.download(ativos_comparacao, start=data_inicial, end=data_final).get('Close')

## Baixando os dados

In [ ]:
precos = vbt.YFData.download(codigo, start=data_inicial, end=data_final).get()
fechamento = vbt.YFData.download(codigo, start=data_inicial, end=data_final).get('Close')

# Parte I - Analisando a Série

In [ ]:
fechamento.vbt.plot().show()

In [ ]:
precos.vbt.ohlcv.plot().show()

## Gráfico comparativo

Com investimento inicial de \$1 (ou R\$ 1)

In [ ]:
vbt.YFData.download(ativos_comparacao,
                    start=data_inicial,
                    end=data_final).plot(column='Close', base=1).show()

## Gráfico Quant Stat

Retorno Cumulativo, Drawdown e Retorno Diário

In [ ]:
warnings.filterwarnings('ignore')

retornos = fechamento.vbt.to_returns()
retornos.vbt.returns.qs.plot_snapshot(title=codigo)

# Analisando Estratégias de Trade

## Média Móvel Simples (SMA)

SMA: Simple Moving Averages

In [ ]:
n_periodos_curta = 20
n_periodos_longa = 50

media_curta = vbt.MA.run(fechamento, n_periodos_curta, short_name='media_curta')
media_longa = vbt.MA.run(fechamento, n_periodos_longa, short_name='media_longa')

fig = fechamento.vbt.plot(trace_kwargs=dict(name='Preço de Fechamento'))
media_curta.ma.vbt.plot(trace_kwargs=dict(name='Média Móvel Curta'), fig=fig)
media_longa.ma.vbt.plot(trace_kwargs=dict(name='Média Móvel Longa'), fig=fig)

fig.show()

## Índice de Força Relativa (RSI)

RSI: Relative Strength Index

In [ ]:
rsi = vbt.RSI.run(fechamento)
rsi.plot().show()

## Estocástico (Stochastics)

In [ ]:
vbt.STOCH.run(precos['High'], precos['Low'], precos['Close']).plot().show()

## Bandas de Bolinger (Bollinger Bands)

In [ ]:
vbt.BBANDS.run(fechamento).plot().show()

# Parte II - Escolhendo Melhor Estratégia

## Testando manualmente

### Teste dentro da amostra (in-sample)

In [ ]:
estrategia_media_movel_curta = 10
estrategia_media_movel_longa = 30

media_movel_curta = vbt.MA.run(fechamento, estrategia_media_movel_curta, short_name='fast_ma')
media_movel_longa = vbt.MA.run(fechamento, estrategia_media_movel_longa, short_name='slow_ma')

entradas = media_movel_curta.ma_crossed_above(media_movel_longa)
saidas = media_movel_curta.ma_crossed_below(media_movel_longa)

# Create the Signals Portfolio
pf = vbt.Portfolio.from_signals(fechamento,
                                entries=entradas,
                                exits=saidas,
                                freq="D",
                                # init_cash=100_000,
                                fees=0.0025,
                                slippage=0.0025)

print("Retorno Total (%):", round(pf.total_return() * 100,2))
print("Retorno Anualizado (%):", round(pf.annualized_return() * 100,2))
print("Índice Sharpe:", round(pf.sharpe_ratio(),2)) # Entenda IS em https://pt.wikipedia.org/wiki/%C3%8Dndice_de_Sharpe

Retorno Total (%): 7.8
Retorno Anualizado (%): 11.54
Índice Sharpe: 0.74


In [ ]:
# print(pf.stats()) # Todas as estatisticas do Portfolio

In [ ]:
# print(pf.returns_stats()) # Todas as estatisticas do retorno

In [ ]:
# Gráfico da Estratégia
fig = pf.close.vbt.plot(trace_kwargs=dict(name='Preço de Fechamento'))
media_movel_curta.ma.vbt.plot(trace_kwargs=dict(name='Média Móvel Curta'), fig=fig)
media_movel_longa.ma.vbt.plot(trace_kwargs=dict(name='Média Móvel Longa'), fig=fig)
pf.positions.plot(close_trace_kwargs=dict(visible=False), fig=fig)
fig.show()

In [ ]:
# Mostrar as ordens
# pf.orders.records_readable

In [ ]:
# Gráfico de Saldo em Caixa e valor dos Ativos investidos
# pf.plot(subplots=['cash', 'assets', 'value']).show()

### Analisar Fora da Amostra (out-of-sample)

> Bloco com recuo



In [ ]:
# Baixar dados fora da amostra
data_inicial2 = data_final 
data_final2 = '2019-01-01'
precos2 = vbt.YFData.download(codigo, start=data_inicial2, end=data_final2).get()
fechamento2 = vbt.YFData.download(codigo, start=data_inicial2, end=data_final2).get('Close')
print(precos2)

                                Open       High        Low      Close  \
Date                                                                    
2018-01-02 05:00:00+00:00  40.451280  40.960012  40.237326  40.950500   
2018-01-03 05:00:00+00:00  41.014685  41.494891  40.879184  40.943367   
2018-01-04 05:00:00+00:00  41.017055  41.238141  40.907703  41.133541   
2018-01-05 05:00:00+00:00  41.231010  41.689817  41.138297  41.601860   
2018-01-08 05:00:00+00:00  41.447346  41.746878  41.347498  41.447346   
...                              ...        ...        ...        ...   
2018-12-24 05:00:00+00:00  35.747694  36.568095  35.371276  35.429188   
2018-12-26 05:00:00+00:00  35.783888  37.938641  35.402643  37.924164   
2018-12-27 05:00:00+00:00  37.603250  37.827656  36.210987  37.678051   
2018-12-28 05:00:00+00:00  38.003798  38.249919  37.291981  37.697353   
2018-12-31 05:00:00+00:00  38.252327  38.452602  37.757674  38.061707   

                              Volume  Dividends  S

In [ ]:
# Testar estratégia fora da amostra
media_movel_curta2 = vbt.MA.run(fechamento2, estrategia_media_movel_curta, short_name='fast_ma')
media_movel_longa2 = vbt.MA.run(fechamento2, estrategia_media_movel_longa, short_name='slow_ma')

entradas2 = media_movel_curta2.ma_crossed_above(media_movel_longa2)
saidas2 = media_movel_curta2.ma_crossed_below(media_movel_longa2)

# Create the Signals Portfolio
pf2 = vbt.Portfolio.from_signals(fechamento2,
                                entries=entradas2,
                                exits=saidas2,
                                freq="D",
                                # init_cash=100_000,
                                fees=0.0025,
                                slippage=0.0025)

In [ ]:
# Comparar desempenho
print("=" * 70,"\n")
print("RESULTADO NA AMOSTRA DE TESTE (in-sample)\n")
print("Retorno Total (%):", round(pf.total_return() * 100,2))
print("Retorno Anualizado (%):", round(pf.annualized_return() * 100,2))
print("Índice Sharpe:", round(pf.sharpe_ratio(),2))
print("=" * 70,"\n")
print("RESULTADO FORA DA AMOSTRA (out-of-sample)\n")
print("Retorno Total (%):", round(pf2.total_return() * 100,2))
print("Retorno Anualizado (%):", round(pf2.annualized_return() * 100,2))
print("Índice Sharpe:", round(pf2.sharpe_ratio(),2))
print("=" * 70)


RESULTADO NA AMOSTRA DE TESTE (in-sample)

Retorno Total (%): 7.8
Retorno Anualizado (%): 11.54
Índice Sharpe: 0.74

RESULTADO FORA DA AMOSTRA (out-of-sample)

Retorno Total (%): -5.46
Retorno Anualizado (%): -7.84
Índice Sharpe: -0.32


### Comentário


Comparando os dois resultados, percebesse que o resultado fora da amostra teve um resultado diferente, pois todos os valores deram negativo, em mostra um invetimento valido (in-sample), com retorno total de 7.8% a 11.54% ao ano, enquanto o outro mostra prejuizo (out-of-smple) dando um retorno total de -5.46% a -7.84% ao ano.Enquanto o ìndice sharp o qual apresenta o índice onde expressa a relação risco x retorno e informa se o fundo oferece rentabilidade compatível com o risco a que o investidor está exposto. Sendo assim o indice sharp mostra um maior risco em relação aos resultados fora da amostra. Estrategias passadas podem sim funcionar no periodo atual, apenas depende de como abordar e estar ciente que as variaeis passadas e atuais podem se diferentes dependendo da ocasião, alterando o resultado final.

## Otimizando computacionalmente

### Escolher janela de teste

In [ ]:
# Definir intervalo de janelas
janelas = np.arange(10, 50)
print(janelas)

[10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49]


In [ ]:
fechamento

Date
2017-01-03 05:00:00+00:00    27.174757
2017-01-04 05:00:00+00:00    27.144341
2017-01-05 05:00:00+00:00    27.282377
2017-01-06 05:00:00+00:00    27.586529
2017-01-09 05:00:00+00:00    27.839209
                               ...    
2017-12-22 05:00:00+00:00    41.604237
2017-12-26 05:00:00+00:00    40.548737
2017-12-27 05:00:00+00:00    40.555870
2017-12-28 05:00:00+00:00    40.669983
2017-12-29 05:00:00+00:00    40.230190
Name: Close, Length: 251, dtype: float64

In [ ]:
# Testar automaticamente diversas combinações de janelas
# fast_ma, slow_ma = vbt.MA.run_combs(price, windows)
media_movel_curta3, media_movel_longa3 = vbt.MA.run_combs(fechamento, janelas)

# entries = fast_ma.ma_crossed_above(slow_ma)
# exits = fast_ma.ma_crossed_below(slow_ma)
entradas3 = media_movel_curta3.ma_crossed_above(media_movel_longa3)
saidas3 = media_movel_curta3.ma_crossed_below(media_movel_longa3)

# portfolio = vbt.Portfolio.from_signals(price, entries, exits, freq='1d', direction='both')
# Create the Signals Portfolio
pf3 = vbt.Portfolio.from_signals(fechamento,
                                entries=entradas3,
                                exits=saidas3,
                                freq="D",
                                # init_cash=100_000,
                                fees=0.0025,
                                slippage=0.0025)

### Resultado in-sample

In [ ]:
# print("Retorno Total (%):\n\n", round(pf3.total_return().sort_values() * 100,2))
# print("Retorno Anualizado (%):\n\n", round(pf3.annualized_return().sort_values() * 100,2))
print("Índice Sharpe:\n\n", round(pf3.sharpe_ratio().sort_values(),2))

Índice Sharpe:

 ma_1_window  ma_2_window
46           48            -1.59
41           42            -1.48
             43            -1.48
46           47            -1.43
43           44            -1.37
                            ... 
16           21             1.07
17           21             1.09
16           23             1.17
17           22             1.22
16           22             1.32
Name: sharpe_ratio, Length: 780, dtype: float64


### Testar fora da amostra (out-of-sample)

Testando a melhor estratégia encontrada na etapa anterior (atulizar números com as melhores janelas).

### A FAZER: Colocar manualmente a melhor estrategia

In [76]:
estrategia_otima_media_movel_curta = 15
estrategia_otima_media_movel_longa = 24

In [77]:
# Rodar teste
media_movel_curta4 = vbt.MA.run(fechamento2, estrategia_otima_media_movel_curta, short_name='fast_ma')
media_movel_longa4 = vbt.MA.run(fechamento2, estrategia_otima_media_movel_longa, short_name='slow_ma')

entradas4 = media_movel_curta4.ma_crossed_above(media_movel_longa4)
saidas4 = media_movel_curta4.ma_crossed_below(media_movel_longa4)

# Create the Signals Portfolio
pf4 = vbt.Portfolio.from_signals(fechamento2,
                                entries=entradas4,
                                exits=saidas4,
                                freq="D",
                                # init_cash=100_000,
                                fees=0.0025,
                                slippage=0.0025)

### Analisar Resultados (out-of-sample)

In [78]:
# Comparar desempenho
print("=" * 70,"\n")
print("RESULTADO NA AMOSTRA DE TESTE (in-sample)\n")
# print("Retorno Total (%):", round(float(pf3.total_return().sort_values().tail(1).values) * 100,2))
# print("Retorno Anualizado (%):", round(float(pf3.annualized_return().sort_values().tail(1).values) * 100,2))
print("Índice Sharpe:", round(float(pf3.sharpe_ratio().sort_values().tail(1).values),2))
print("=" * 70,"\n")
print("RESULTADO FORA DA AMOSTRA (out-of-sample)\n")
# print("Retorno Total (%):", round(pf4.total_return() * 100,2))
# print("Retorno Anualizado (%):", round(pf4.annualized_return() * 100,2))
print("Índice Sharpe:", round(pf4.sharpe_ratio(),2))
print("=" * 70)


RESULTADO NA AMOSTRA DE TESTE (in-sample)

Índice Sharpe: 1.32

RESULTADO FORA DA AMOSTRA (out-of-sample)

Índice Sharpe: 0.52


### COMENTÁRIO: 

(Funcinou? Não funcinou? Porque deu diferente?)

Não funcionou. Em minhas alterações da media movel curta e lunga, os resultados fora da amostra (out-of-sample) se manteve abaixo dos resultados na amostra (in-sample). Ao iniciar os testecom amedia movel curta em 16 e a media movel longa em 22 o indice sharp estava resultando em -17, trazzendo um aior risco de investimento, após diversas alterações consegui chegar em em um numero mais aceitavel do indice sharp, aletrando sua meida movel curta de 16 para 15 e sua media longa de 22 pra 24 assim resultou em um indice de 0,52, muio superior comparado ao anterior. Está dando diferente pois os resultados fora da amostra nãos estão conseguindo se igualar aos resultados já adquiridos, provando-se inferiror nesse cenario.

# Próximo tópico:

Treino, validação e teste.

<img src="https://vectorbt.dev/assets/images/features_rolling_split.svg">
